In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import gc
import joblib # Optional, kept for consistency
import traceback
import warnings
import re # For feature filtering if needed
import time # Import time for timing
import shap # Import SHAP
import os # Import os for path manipulation

# Use standard tqdm if not in notebook environment
try:
    from tqdm import tqdm # Use standard tqdm for loops
except ImportError:
    # Fallback if tqdm is not installed (less pretty output)
    def tqdm(iterable, *args, **kwargs):
        if hasattr(iterable, '__len__') and len(iterable) > 0:
            print("Warning: tqdm library not found. Progress bars will not be shown.")
        return iterable

# Scikit-learn imports
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    average_precision_score,
    roc_auc_score,
    f1_score,
    precision_score,
    recall_score,
    accuracy_score
)
# Import matplotlib for saving SHAP plots
import matplotlib.pyplot as plt

# Suppress warnings for cleaner output (optional)
warnings.filterwarnings('ignore')

# ==============================================================
# --- Configuration ---
# ==============================================================

# !! IMPORTANT: Verify this BASE_PATH is correct !!
BASE_PATH = Path(r'C:\Users\sdogan') # <--- CHANGE THIS TO YOUR ACTUAL BASE PATH
# Or use forward slashes:
# BASE_PATH = Path('/path/to/your/base/directory')

DATA_DIR = BASE_PATH / 'Reddit_Virality_Data'
API_RESULTS_DIR = DATA_DIR / 'API_results'
NET_TEMP_PROCESSED_DIR = API_RESULTS_DIR / 'preprocessed_data'
EVALUATION_RESULTS_DIR = API_RESULTS_DIR / 'evaluation_results'
FEATURE_IMPORTANCE_DIR = EVALUATION_RESULTS_DIR / 'feature_importances'
SHAP_RESULTS_DIR = EVALUATION_RESULTS_DIR / 'shap_results'

TIME_WINDOWS_TO_PROCESS = [ 30, 60, 120, 180, 240, 300, 360, 420 ]

TARGET_COLUMN = 'is_viral'
ID_COLUMN = 'id'
ORIGINAL_ID_COLUMN = 'original_id'
RANDOM_STATE = 42
SNAPSHOT_TIME_COLUMN = 'snapshot_time_minutes'

INTERMEDIATE_TRAIN_PREFIX = "train_"
INTERMEDIATE_VALID_PREFIX = "val_"
INTERMEDIATE_TEST_PREFIX = "test_"

FINAL_RESULTS_FILENAME = "RF_LR_baseline_all_windows_RF_ablation_60min.csv"

MODALITY_PREFIX_GROUPS = {
    "contextual": ["contextual_"],
    "temporal": ["temporal_"],
    "visual": ["visual_"],
    "network": ["network_"],
    "textual": ["textual_"],
}

print(f"--- Testing RF & LR Baseline (All Windows) & RF Ablation (60min Window) ---")
print(f"--- Time Windows: {TIME_WINDOWS_TO_PROCESS} ---")
print(f"--- Using {len(MODALITY_PREFIX_GROUPS)} Modalities for RF Ablation (Custom Temporal Logic) ---")
print(f"Reading intermediate processed data from: {NET_TEMP_PROCESSED_DIR}")
print(f"Saving final comparison results to: {EVALUATION_RESULTS_DIR}")
print(f"Saving baseline feature importance results to: {FEATURE_IMPORTANCE_DIR}")
print(f"Saving baseline RF SHAP summary results and plots to: {SHAP_RESULTS_DIR}")

EVALUATION_RESULTS_DIR.mkdir(parents=True, exist_ok=True)
FEATURE_IMPORTANCE_DIR.mkdir(parents=True, exist_ok=True)
SHAP_RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# ==============================================================
# --- Helper Function for Saving Importance ---
# ==============================================================
def save_feature_importance(model, feature_names, model_type, window_minutes, save_dir, scenario="baseline"):
    """
    Calculates, sorts, and saves feature importance/coefficients.
    Only saves if scenario is "baseline".
    """
    if scenario != "baseline":
        return

    importance_df = None
    col_name = 'importance'
    try:
        if hasattr(model, 'feature_importances_'): # For RF
            importances = model.feature_importances_
            importance_df = pd.DataFrame({'feature': feature_names, col_name: importances})
            importance_df = importance_df.sort_values(by=col_name, ascending=False)
        elif hasattr(model, 'coef_'): # For LR
            col_name = 'coefficient'
            if model.coef_.ndim == 2 and model.coef_.shape[0] == 1:
                coefficients = model.coef_[0]
                importance_df = pd.DataFrame({'feature': feature_names, col_name: coefficients})
                importance_df = importance_df.iloc[importance_df[col_name].abs().argsort()[::-1]]
            else:
                print(f"      LR coefficient shape unexpected ({model.coef_.shape}). Skipping importance save.")
                return
        else:
            print(f"      Model type {model_type} does not have standard importance/coefficients. Skipping.")
            return

        if importance_df is not None:
            filename = f"{model_type}_feature_importance_{window_minutes}min_baseline.csv"
            save_path = save_dir / filename
            importance_df.to_csv(save_path, index=False, float_format='%.6f')
            # print(f"      Saved baseline feature importance/coefficients to: {save_path}")

    except Exception as e:
        print(f"      Error saving feature importance for {model_type}, window {window_minutes} min, scenario {scenario}: {e}")
        traceback.print_exc()

# ==============================================================
# --- Helper Function for Saving SHAP Summary (RF ONLY) ---
# ==============================================================
def save_shap_summary(shap_values, feature_names, model_type, window_minutes, save_dir, scenario="baseline"):
    """
    Saves mean absolute SHAP values for RF model in baseline scenario.
    """
    if scenario != "baseline" or model_type != 'RF':
        return

    try:
        mean_abs_shap = np.abs(shap_values).mean(axis=0)
        if len(mean_abs_shap) != len(feature_names):
            print(f"      Error: Mismatch SHAP values ({len(mean_abs_shap)}) and feature names ({len(feature_names)}). Skipping SHAP save.")
            return

        shap_summary_df = pd.DataFrame({'feature': feature_names, 'mean_abs_shap': mean_abs_shap})
        shap_summary_df = shap_summary_df.sort_values(by='mean_abs_shap', ascending=False)
        
        filename = f"RF_shap_summary_{window_minutes}min_baseline.csv"
        save_path = save_dir / filename
        shap_summary_df.to_csv(save_path, index=False, float_format='%.6f')
        # print(f"      Saved baseline RF SHAP summary CSV to: {save_path}")

    except Exception as e:
        print(f"      Error calculating/saving RF SHAP summary for window {window_minutes} min, scenario {scenario}: {e}")
        traceback.print_exc()

# ==============================================================
# --- Helper Function for Evaluation ---
# ==============================================================
def evaluate_model(model, X_eval, y_eval, model_type, window, scenario, eval_set_name):
    """
    Evaluates a trained model.
    """
    metrics = {}
    # print(f"        Evaluating {model_type} on {eval_set_name.capitalize()} Set ({scenario}, {window} min)...")
    try:
        probs = model.predict_proba(X_eval)[:, 1]
        preds = (probs > 0.5).astype(int)

        metrics[f'pr_auc_{eval_set_name}'] = average_precision_score(y_eval, probs)
        metrics[f'roc_auc_{eval_set_name}'] = roc_auc_score(y_eval, probs)
        metrics[f'f1_score_{eval_set_name}'] = f1_score(y_eval, preds, zero_division=0)
        metrics[f'precision_{eval_set_name}'] = precision_score(y_eval, preds, zero_division=0)
        metrics[f'recall_{eval_set_name}'] = recall_score(y_eval, preds, zero_division=0)
        metrics[f'accuracy_{eval_set_name}'] = accuracy_score(y_eval, preds)
        # print(f"          {model_type} {eval_set_name.capitalize()} PR AUC: {metrics[f'pr_auc_{eval_set_name}']:.5f}, ROC AUC: {metrics[f'roc_auc_{eval_set_name}']:.5f}")

    except Exception as e:
        print(f"        Error during {model_type} evaluation on {eval_set_name} for scenario '{scenario}', window {window} min: {e}")
        traceback.print_exc()
        for metric_name in ['pr_auc', 'roc_auc', 'f1_score', 'precision', 'recall', 'accuracy']:
            metrics[f'{metric_name}_{eval_set_name}'] = np.nan
        metrics[f'error_{eval_set_name}'] = str(e)
    return metrics

# ==============================================================
# --- Main Script Logic ---
# ==============================================================
if __name__ == "__main__":
    all_results_list = []
    script_start_time = time.time()

    for window_minutes in tqdm(TIME_WINDOWS_TO_PROCESS, desc="Processing Time Windows", unit="window"):
        window_start_time = time.time()
        print(f"\n===== Processing Time Window: {window_minutes} minutes =====")

        train_filename = f"{INTERMEDIATE_TRAIN_PREFIX}{window_minutes}min.parquet"
        valid_filename = f"{INTERMEDIATE_VALID_PREFIX}{window_minutes}min.parquet"
        test_filename = f"{INTERMEDIATE_TEST_PREFIX}{window_minutes}min.parquet"
        intermediate_train_path = NET_TEMP_PROCESSED_DIR / train_filename
        intermediate_valid_path = NET_TEMP_PROCESSED_DIR / valid_filename
        intermediate_test_path = NET_TEMP_PROCESSED_DIR / test_filename

        df_train, df_valid, df_test = None, None, None
        try:
            print(f"  Loading data for window: {window_minutes} min...")
            df_train = pd.read_parquet(intermediate_train_path)
            df_valid = pd.read_parquet(intermediate_valid_path)
            df_test = pd.read_parquet(intermediate_test_path)
            print(f"    Loaded Train: {df_train.shape}, Valid: {df_valid.shape}, Test: {df_test.shape}")
        except FileNotFoundError as e:
            print(f"  WARNING: Data file not found for window {window_minutes} min ({e}). Skipping.")
            error_msg = 'InputFileNotFound'
            all_results_list.append({'model_type': 'RandomForestClassifier', 'time_window': window_minutes, 'ablation_scenario': 'baseline', 'error': error_msg})
            all_results_list.append({'model_type': 'LogisticRegression', 'time_window': window_minutes, 'ablation_scenario': 'baseline', 'error': error_msg})
            if window_minutes == 60:
                for modality_name in MODALITY_PREFIX_GROUPS.keys():
                    all_results_list.append({'model_type': 'RandomForestClassifier', 'time_window': window_minutes, 'ablation_scenario': f'exclude_{modality_name}', 'error': error_msg})
            if 'df_train' in locals(): del df_train; gc.collect()
            if 'df_valid' in locals(): del df_valid; gc.collect()
            if 'df_test' in locals(): del df_test; gc.collect()
            continue
        except Exception as e:
            print(f"  ERROR loading data for window {window_minutes} min: {e}. Skipping.")
            traceback.print_exc()
            error_msg = f'DataLoadError: {e}'
            all_results_list.append({'model_type': 'RandomForestClassifier', 'time_window': window_minutes, 'ablation_scenario': 'baseline', 'error': error_msg})
            all_results_list.append({'model_type': 'LogisticRegression', 'time_window': window_minutes, 'ablation_scenario': 'baseline', 'error': error_msg})
            if window_minutes == 60:
                for modality_name in MODALITY_PREFIX_GROUPS.keys():
                    all_results_list.append({'model_type': 'RandomForestClassifier', 'time_window': window_minutes, 'ablation_scenario': f'exclude_{modality_name}', 'error': error_msg})
            if 'df_train' in locals(): del df_train; gc.collect()
            if 'df_valid' in locals(): del df_valid; gc.collect()
            if 'df_test' in locals(): del df_test; gc.collect()
            continue

        X_train, y_train, X_valid, y_valid, X_test, y_test = None, None, None, None, None, None
        X_train_scaled, X_valid_scaled, X_test_scaled = None, None, None
        all_feature_cols = []
        modality_feature_sets = {name: [] for name in MODALITY_PREFIX_GROUPS.keys()}
        scaler = None

        try:
            print("  Preparing features (X), target (y), and modality groups...")
            original_id_col_to_use = None
            if ORIGINAL_ID_COLUMN in df_train.columns: original_id_col_to_use = ORIGINAL_ID_COLUMN
            elif ID_COLUMN in df_train.columns: original_id_col_to_use = ID_COLUMN
            
            if 'media_type' in df_train.columns and original_id_col_to_use:
                audio_extensions = ['.m4a', '.mp3', '.wav', '.aac', '.ogg', '.flac']
                for df_split_name, df_split in [("Train", df_train), ("Valid", df_valid), ("Test", df_test)]:
                    if 'media_type' in df_split.columns and original_id_col_to_use in df_split.columns:
                        df_split['media_type'] = df_split['media_type'].fillna('Unknown').astype(str)
                        df_split[original_id_col_to_use] = df_split[original_id_col_to_use].fillna('').astype(str)
                        unknown_media_mask = df_split['media_type'].str.lower().isin(['unknown', ''])
                        is_audio_mask = df_split[original_id_col_to_use].str.lower().apply(lambda x: any(x.endswith(ext) for ext in audio_extensions))
                        update_mask = unknown_media_mask & is_audio_mask
                        if update_mask.sum() > 0:
                            df_split.loc[update_mask, 'media_type'] = 'audio'
                            # print(f"      Updated {update_mask.sum()} 'Unknown' media types to 'audio' in {df_split_name} based on extension.")

            exclude_cols = [TARGET_COLUMN, ID_COLUMN, SNAPSHOT_TIME_COLUMN, ORIGINAL_ID_COLUMN, 'media_type'] # Add media_type if it was used for inference
            initial_feature_cols = [col for col in df_train.columns if col not in exclude_cols and col not in MODALITY_PREFIX_GROUPS.keys()] # Avoid picking up modality group names if they are columns
            
            common_features = set(initial_feature_cols).intersection(df_valid.columns).intersection(df_test.columns)
            all_feature_cols = [col for col in initial_feature_cols if col in common_features]

            if not all(TARGET_COLUMN in df.columns for df in [df_train, df_valid, df_test]):
                raise ValueError(f"Target column '{TARGET_COLUMN}' not found.")

            X_train = df_train[all_feature_cols].fillna(0)
            y_train = df_train[TARGET_COLUMN]
            X_valid = df_valid[all_feature_cols].fillna(0)
            y_valid = df_valid[TARGET_COLUMN]
            X_test = df_test[all_feature_cols].fillna(0)
            y_test = df_test[TARGET_COLUMN]
            print(f"    Using {len(all_feature_cols)} common features.")

            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_valid_scaled = scaler.transform(X_valid)
            X_test_scaled = scaler.transform(X_test)

            temporal_suffix = f"_0to{window_minutes}min"
            assigned_features = set()
            temporal_features = {col for col in all_feature_cols if col.startswith('temporal_') or col.endswith(temporal_suffix)}
            modality_feature_sets['temporal'] = list(temporal_features)
            assigned_features.update(temporal_features)

            for name, prefixes in MODALITY_PREFIX_GROUPS.items():
                if name == 'temporal': continue
                features_in_group = set()
                for prefix in prefixes:
                    matched_cols = {col for col in all_feature_cols if col.startswith(prefix) and col not in assigned_features}
                    features_in_group.update(matched_cols)
                modality_feature_sets[name] = list(features_in_group)
                assigned_features.update(features_in_group)
            
            unassigned = [col for col in all_feature_cols if col not in assigned_features]
            if unassigned: print(f"    WARNING: {len(unassigned)} features unassigned to modalities: {unassigned[:5]}...")
            
            del df_train, df_valid, df_test; gc.collect()

        except Exception as e:
            print(f"  ERROR preparing data/modalities for window {window_minutes} min: {e}. Skipping.")
            traceback.print_exc()
            error_msg = f'PrepareDataError: {e}'
            all_results_list.append({'model_type': 'RandomForestClassifier', 'time_window': window_minutes, 'ablation_scenario': 'baseline', 'error': error_msg})
            all_results_list.append({'model_type': 'LogisticRegression', 'time_window': window_minutes, 'ablation_scenario': 'baseline', 'error': error_msg})
            if window_minutes == 60:
                for modality_name in MODALITY_PREFIX_GROUPS.keys():
                    all_results_list.append({'model_type': 'RandomForestClassifier', 'time_window': window_minutes, 'ablation_scenario': f'exclude_{modality_name}', 'error': error_msg})
            if 'df_train' in locals(): del df_train; gc.collect()
            if 'df_valid' in locals(): del df_valid; gc.collect()
            if 'df_test' in locals(): del df_test; gc.collect()
            if 'X_train' in locals(): del X_train, y_train, X_valid, y_valid, X_test, y_test, X_train_scaled, X_valid_scaled, X_test_scaled, scaler, all_feature_cols, modality_feature_sets; gc.collect()
            continue
        
        # Check if data preparation was successful before proceeding to scenarios
        if X_train is None or X_train_scaled is None:
            print(f"  ERROR: X data splits not available after preparation for window {window_minutes} min. Skipping model training for this window.")
            # Error already logged if exception occurred, this is a fallback
            error_msg = 'X_data_missing_post_successful_prep_block'
            all_results_list.append({'model_type': 'RandomForestClassifier', 'time_window': window_minutes, 'ablation_scenario': 'baseline', 'error': error_msg})
            all_results_list.append({'model_type': 'LogisticRegression', 'time_window': window_minutes, 'ablation_scenario': 'baseline', 'error': error_msg})
            if window_minutes == 60:
                for modality_name in MODALITY_PREFIX_GROUPS.keys():
                    all_results_list.append({'model_type': 'RandomForestClassifier', 'time_window': window_minutes, 'ablation_scenario': f'exclude_{modality_name}', 'error': error_msg})
            del X_train, y_train, X_valid, y_valid, X_test, y_test, X_train_scaled, X_valid_scaled, X_test_scaled, scaler, all_feature_cols, modality_feature_sets; gc.collect()
            continue


        scenarios_to_run = ["baseline"]
        rf_ablation_active = False
        if window_minutes == 60:
            scenarios_to_run.extend([f"exclude_{name}" for name in MODALITY_PREFIX_GROUPS.keys()])
            rf_ablation_active = True
            print(f"\n  >>> RF Ablation Study ACTIVE for {window_minutes} min window <<<")
        else:
            print(f"\n  >>> Baseline Only for {window_minutes} min window <<<")

        for scenario in scenarios_to_run:
            scenario_start_time = time.time()
            print(f"\n  --- Running Scenario: {scenario} (Window {window_minutes} min) ---")

            current_feature_cols_scenario = list(all_feature_cols)
            if scenario != "baseline":
                modality_to_exclude = scenario.split("exclude_")[-1]
                features_to_exclude = modality_feature_sets.get(modality_to_exclude, [])
                if features_to_exclude:
                    current_feature_cols_scenario = [col for col in all_feature_cols if col not in features_to_exclude]
                    print(f"    Excluding {len(features_to_exclude)} '{modality_to_exclude}' features. Using {len(current_feature_cols_scenario)} features.")
                else:
                    print(f"    WARNING: No features for modality '{modality_to_exclude}' to exclude. Using all features.")
            else:
                 print(f"    Using all {len(current_feature_cols_scenario)} features (baseline).")

            if not current_feature_cols_scenario:
                print(f"    ERROR: Feature list for scenario '{scenario}' is empty. Skipping training.")
                if scenario == "baseline" or (rf_ablation_active and scenario.startswith("exclude_")):
                    all_results_list.append({'model_type': 'RandomForestClassifier', 'time_window': window_minutes, 'ablation_scenario': scenario, 'error': 'EmptyFeatureList'})
                if scenario == "baseline":
                    all_results_list.append({'model_type': 'LogisticRegression', 'time_window': window_minutes, 'ablation_scenario': scenario, 'error': 'EmptyFeatureList'})
                continue

            X_train_scenario = X_train[current_feature_cols_scenario]
            X_valid_scenario = X_valid[current_feature_cols_scenario]
            X_test_scenario = X_test[current_feature_cols_scenario]
            
            feature_indices = [all_feature_cols.index(col) for col in current_feature_cols_scenario]
            X_train_scaled_scenario = X_train_scaled[:, feature_indices]
            X_valid_scaled_scenario = X_valid_scaled[:, feature_indices]
            X_test_scaled_scenario = X_test_scaled[:, feature_indices]

            # === Random Forest ===
            if scenario == "baseline" or (rf_ablation_active and scenario.startswith("exclude_")):
                print("\n    --- Random Forest ---")
                rf_model = RandomForestClassifier(n_estimators=250, class_weight='balanced', random_state=RANDOM_STATE, n_jobs=-1, max_depth=20, min_samples_leaf=10, oob_score=False, verbose=0)
                rf_results = {'model_type': 'RandomForestClassifier', 'time_window': window_minutes, 'ablation_scenario': scenario}
                try:
                    print(f"      Training RF model...")
                    rf_model.fit(X_train_scenario, y_train)
                    
                    valid_metrics_rf = evaluate_model(rf_model, X_valid_scenario, y_valid, 'RF', window_minutes, scenario, 'valid')
                    rf_results.update(valid_metrics_rf)
                    test_metrics_rf = evaluate_model(rf_model, X_test_scenario, y_test, 'RF', window_minutes, scenario, 'test')
                    rf_results.update(test_metrics_rf)
                    rf_results['error'] = None
                    print(f"      RF ({scenario}): Test PR AUC: {rf_results.get('pr_auc_test', np.nan):.4f}, ROC AUC: {rf_results.get('roc_auc_test', np.nan):.4f}")

                    save_feature_importance(rf_model, current_feature_cols_scenario, 'RF', window_minutes, FEATURE_IMPORTANCE_DIR, scenario)

                    if scenario == "baseline": # SHAP only for baseline RF
                        print(f"      Calculating SHAP values for baseline RF model...")
                        explainer = shap.TreeExplainer(rf_model)
                        X_valid_shap_df = pd.DataFrame(X_valid_scenario, columns=current_feature_cols_scenario) # SHAP prefers DataFrame
                        shap_values = explainer.shap_values(X_valid_shap_df)
                        positive_class_index = np.where(rf_model.classes_ == 1)[0][0]
                        shap_values_positive = shap_values[positive_class_index]
                        
                        save_shap_summary(shap_values_positive, current_feature_cols_scenario, 'RF', window_minutes, SHAP_RESULTS_DIR, scenario)
                        
                        shap.summary_plot(shap_values_positive, X_valid_shap_df, plot_type="bar", show=False)
                        plt.title(f"SHAP Importance (Mean Abs) - RF Baseline {window_minutes}min")
                        plt.tight_layout(); plt.savefig(SHAP_RESULTS_DIR / f"RF_shap_summary_bar_{window_minutes}min_baseline.png", bbox_inches='tight'); plt.close()
                        
                        shap.summary_plot(shap_values_positive, X_valid_shap_df, show=False)
                        plt.title(f"SHAP Detailed Summary - RF Baseline {window_minutes}min")
                        plt.tight_layout(); plt.savefig(SHAP_RESULTS_DIR / f"RF_shap_summary_beeswarm_{window_minutes}min_baseline.png", bbox_inches='tight'); plt.close()
                        del explainer, shap_values, shap_values_positive, X_valid_shap_df; gc.collect()

                except Exception as e:
                    print(f"      Error during RF for scenario '{scenario}', window {window_minutes} min: {e}")
                    traceback.print_exc()
                    rf_results['error'] = f'TrainEvalError: {e}'
                finally:
                    all_results_list.append(rf_results)
                    if 'rf_model' in locals(): del rf_model; gc.collect()
            
            # === Logistic Regression ===
            if scenario == "baseline":
                print("\n    --- Logistic Regression ---")
                lr_model = LogisticRegression(class_weight='balanced', random_state=RANDOM_STATE, solver='liblinear', max_iter=1000)
                lr_results = {'model_type': 'LogisticRegression', 'time_window': window_minutes, 'ablation_scenario': scenario}
                try:
                    print(f"      Training LR model...")
                    lr_model.fit(X_train_scaled_scenario, y_train)

                    valid_metrics_lr = evaluate_model(lr_model, X_valid_scaled_scenario, y_valid, 'LR', window_minutes, scenario, 'valid')
                    lr_results.update(valid_metrics_lr)
                    test_metrics_lr = evaluate_model(lr_model, X_test_scaled_scenario, y_test, 'LR', window_minutes, scenario, 'test')
                    lr_results.update(test_metrics_lr)
                    lr_results['error'] = None
                    print(f"      LR ({scenario}): Test PR AUC: {lr_results.get('pr_auc_test', np.nan):.4f}, ROC AUC: {lr_results.get('roc_auc_test', np.nan):.4f}")
                    
                    save_feature_importance(lr_model, current_feature_cols_scenario, 'LR', window_minutes, FEATURE_IMPORTANCE_DIR, scenario)

                except Exception as e:
                    print(f"      Error during LR for scenario '{scenario}', window {window_minutes} min: {e}")
                    traceback.print_exc()
                    lr_results['error'] = f'TrainEvalError: {e}'
                finally:
                    all_results_list.append(lr_results)
                    if 'lr_model' in locals(): del lr_model; gc.collect()

            del X_train_scenario, X_valid_scenario, X_test_scenario
            del X_train_scaled_scenario, X_valid_scaled_scenario, X_test_scaled_scenario
            gc.collect()
            print(f"  --- Finished scenario '{scenario}'. Took {time.time() - scenario_start_time:.2f} seconds ---")
            # --- End of scenario loop ---

        if 'X_train' in locals(): del X_train, y_train, X_valid, y_valid, X_test, y_test; gc.collect()
        if 'X_train_scaled' in locals(): del X_train_scaled, X_valid_scaled, X_test_scaled; gc.collect()
        if 'scaler' in locals(): del scaler; gc.collect()
        if 'all_feature_cols' in locals(): del all_feature_cols; gc.collect()
        if 'modality_feature_sets' in locals(): del modality_feature_sets; gc.collect()
        
        print(f"----- Finished window {window_minutes} min | Total Time: {time.time() - window_start_time:.2f} seconds -----")
        # --- End of window loop ---

    print("\n" + "="*60)
    print(f"===== Processing Final RF & LR (Baseline All Windows), RF (Ablation 60min) Results =====")
    print("="*60 + "\n")

    if all_results_list:
        df_final_results = pd.DataFrame(all_results_list)
        column_order = [
            'time_window', 'ablation_scenario', 'model_type',
            'pr_auc_valid', 'roc_auc_valid', 'f1_score_valid', 'precision_valid', 'recall_valid', 'accuracy_valid',
            'pr_auc_test', 'roc_auc_test', 'f1_score_test', 'precision_test', 'recall_test', 'accuracy_test',
            'error'
        ]
        available_cols = [c for c in column_order if c in df_final_results.columns]
        error_cols = [c for c in df_final_results.columns if c.startswith('error_') and c != 'error']
        final_cols = available_cols + error_cols
        df_final_results = df_final_results[final_cols]

        scenario_order_list = ['baseline'] + [f"exclude_{name}" for name in MODALITY_PREFIX_GROUPS.keys()] + ['error_scenario']
        df_final_results['ablation_scenario'] = df_final_results['ablation_scenario'].fillna('error_scenario')
        df_final_results['ablation_scenario_cat'] = pd.Categorical(
            df_final_results['ablation_scenario'], categories=scenario_order_list, ordered=True
        )
        
        df_final_results_sorted = df_final_results.sort_values(
            by=['time_window', 'ablation_scenario_cat', 'model_type', 'pr_auc_test'],
            ascending=[True, True, True, False],
            na_position='last'
        ).drop(columns=['ablation_scenario_cat'])

        print("\n--- Aggregated Results Table ---")
        print(df_final_results_sorted.to_string(index=False, float_format='%.4f'))

        final_results_save_path = EVALUATION_RESULTS_DIR / FINAL_RESULTS_FILENAME
        try:
            df_final_results_sorted.to_csv(final_results_save_path, index=False, float_format='%.5f')
            print(f"\nSaved aggregated results table to: {final_results_save_path}")
        except Exception as e:
            print(f"Error saving aggregated results table: {e}")
            traceback.print_exc()
    else:
        print("\nNo results were generated.")

    script_duration = time.time() - script_start_time
    print(f"\n--- Full Script Finished | Total Time: {script_duration:.2f} seconds ({script_duration/60:.2f} minutes) ---")
    print(f"--- Baseline feature importance files saved in: {FEATURE_IMPORTANCE_DIR} ---")
    print(f"--- Baseline RF SHAP summary CSVs and plots saved in: {SHAP_RESULTS_DIR} ---")

